# Ripple Detection
Ripple detection depends on a set of LFPs, the parameters used for detection and the speed of the animal. You will need `RippleLFPSelection`, `RippleParameters`, and `IntervalPositionInfo` to be populated accordingly. Let's import these:

In [1]:
from spyglass.common.common_ripple import (
    RippleLFPSelection,
    RippleParameters,
    RippleTimes,
)
from spyglass.common import IntervalPositionInfo

[2022-11-28 14:37:02,551][INFO]: Connecting edeno@lmf-db.cin.ucsf.edu:3306
[2022-11-28 14:37:02,627][INFO]: Connected edeno@lmf-db.cin.ucsf.edu:3306
/home/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/position_tools/core.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


## 1. Selecting Electrodes

First let's pick which electrodes we want to do ripple detection on. This involves the calling the method `set_lfp_electrodes` from `RippleLFPSelection`

In [2]:
?RippleLFPSelection.set_lfp_electrodes

Signature:
RippleLFPSelection.set_lfp_electrodes(
    nwb_file_name,
    electrode_list,
    group_name='CA1',
)
Docstring:
Removes all electrodes for the specified nwb file and then adds back the electrodes in the list

Parameters
----------
nwb_file_name : str
    The name of the nwb file for the desired session
electrode_list : list
    list of electrodes to be used for LFP
group_name : str, optional
File:      /cumulus/edeno/spyglass/src/spyglass/common/common_ripple.py
Type:      function


From the function parameters, we can see that we need the `nwb_file_name`, a list of electrodes (`electrode_list`), and to set a `group_name`. 

### How to set the `group__name`?

By default `group_name` is set to CA1 because this is what is commonly used to detect ripples. But if you had a different set of electrodes you wanted to detect on (say on PFC), you could set a different name.

### How do we figure out the `electrode_list`?

You should already know which animal and session you want to choose, so the next question is how you set the `electrode_list`. Let's first define the nwb file we want to look at.

In [3]:
nwb_file_name = "chimi20200216_new_.nwb"

Now we can look at `electrode_id` in the `Electrode` table:

In [4]:
from spyglass.common import Electrode, BrainRegion

electrodes = (
    (Electrode() & {"nwb_file_name": nwb_file_name}) * BrainRegion
).fetch(format="frame")
electrodes

probe_type  \
nwb_file_name          electrode_group_name electrode_id region_id                 
chimi20200216_new_.nwb 0                    0            6          tetrode_12.5   
                                            1            6          tetrode_12.5   
                                            2            6          tetrode_12.5   
                                            3            6          tetrode_12.5   
                       1                    4            6          tetrode_12.5   
...                                                                          ...   
                       8                    35           6          tetrode_12.5   
                       9                    36           6          tetrode_12.5   
                                            37           6          tetrode_12.5   
                                            38           6          tetrode_12.5   
                                            39           6          tetrode_12.5   

                                                                   probe_shank  \
nwb_file_name          electrode_group_name electrode_id region_id               
chimi20200216_new_.nwb 0                    0            6                   0   
                                            1            6                   0   
                                            2            6                   0   
                                            3            6                   0   
                       1                    4            6                   0   
...                                                                        ...   
                       8                    35           6                   0   
                       9                    36           6                   0   
                                            37           6                   0   
                                            38           6                   0   
                                            39           6                   0   

                                                                   probe_electrode  \
nwb_file_name          electrode_group_name electrode_id region_id                   
chimi20200216_new_.nwb 0                    0            6                       0   
                                            1            6                       1   
                                            2            6                       2   
                                            3            6                       3   
                       1                    4            6                       0   
...                                                                            ...   
                       8                    35           6                       3   
                       9                    36           6                       0   
                                            37           6                       1   
                                            38           6                       2   
                                            39           6                       3   

                                                                   name  \
nwb_file_name          electrode_group_name electrode_id region_id        
chimi20200216_new_.nwb 0                    0            6            0   
                                            1            6            1   
                                            2            6            2   
                                            3            6            3   
                       1                    4            6            4   
...                                                                 ...   
                       8                    35           6           35   
                       9                    36           6           36   
                                         

For ripple detection, we want only tetrodes, and only the first good wire on each tetrode. We will assume that is the first wire on each tetrode. I will do this using pandas syntax but you could use datajoint to filter this table as well. Here is the filtered table.

In [5]:
electrodes.loc[
    (electrodes.region_name == "Hippocampus")
    & (electrodes.probe_electrode == 0)
]

probe_type  \
nwb_file_name          electrode_group_name electrode_id region_id                 
chimi20200216_new_.nwb 0                    0            6          tetrode_12.5   
                       1                    4            6          tetrode_12.5   
                       10                   40           6          tetrode_12.5   
                       11                   44           6          tetrode_12.5   
                       17                   68           6          tetrode_12.5   
                       18                   72           6          tetrode_12.5   
                       19                   76           6          tetrode_12.5   
                       2                    8            6          tetrode_12.5   
                       21                   84           6          tetrode_12.5   
                       22                   88           6          tetrode_12.5   
                       23                   92           6          tetrode_12.5   
                       3                    12           6          tetrode_12.5   
                       5                    20           6          tetrode_12.5   
                       6                    24           6          tetrode_12.5   
                       7                    28           6          tetrode_12.5   
                       8                    32           6          tetrode_12.5   
                       9                    36           6          tetrode_12.5   

                                                                   probe_shank  \
nwb_file_name          electrode_group_name electrode_id region_id               
chimi20200216_new_.nwb 0                    0            6                   0   
                       1                    4            6                   0   
                       10                   40           6                   0   
                       11                   44           6                   0   
                       17                   68           6                   0   
                       18                   72           6                   0   
                       19                   76           6                   0   
                       2                    8            6                   0   
                       21                   84           6                   0   
                       22                   88           6                   0   
                       23                   92           6                   0   
                       3                    12           6                   0   
                       5                    20           6                   0   
                       6                    24           6                   0   
                       7                    28           6                   0   
                       8                    32           6                   0   
                       9                    36           6                   0   

                                                                   probe_electrode  \
nwb_file_name          electrode_group_name electrode_id region_id                   
chimi20200216_new_.nwb 0                    0            6                       0   
                       1                    4            6                       0   
                       10                   40           6                       0   
                       11                   44           6                       0   
                       17                   68           6                       0   
                       18                   72           6                       0   
                       19                   76           6                       0   
                       2                    8            6                       0   
                       21                   84        

We only want the electrode_id to put in the `electrode_list`:

In [6]:
electrode_list = (
    electrodes.loc[
        (electrodes.region_name == "Hippocampus")
        & (electrodes.probe_electrode == 0)
    ]
    .reset_index()
    .electrode_id
).tolist()

electrode_list

[0, 4, 40, 44, 68, 72, 76, 8, 84, 88, 92, 12, 20, 24, 28, 32, 36]

Now we can set `RippleLFPSelection`.

In [7]:
RippleLFPSelection.set_lfp_electrodes(
    nwb_file_name,
    electrode_list,
    group_name="CA1",
)

In [8]:
RippleLFPSelection()

nwb_file_name name of the NWB file,group_name
chimi20200216_new_.nwb,CA1


## 2. Setting the Ripple Parameters

Now that we know which electrodes to use, we can set up the ripple detection parameters:

In [10]:
RippleParameters()

ripple_param_name a name for this set of parameters,ripple_param_dict dictionary of parameters
default,=BLOB=


Let's look at the default ripple parameters:

In [14]:
(RippleParameters() & {"ripple_param_name": "default"}).fetch1()

{'ripple_param_name': 'default',
 'ripple_param_dict': {'filter_name': 'Ripple 150-250 Hz',
  'speed_name': 'head_speed',
  'ripple_detection_algorithm': 'Kay_ripple_detector',
  'ripple_detection_params': {'speed_threshold': 4.0,
   'minimum_duration': 0.015,
   'zscore_threshold': 2.0,
   'smoothing_sigma': 0.004,
   'close_ripple_threshold': 0.0}}}

+ `filter_name` refers to which bandpass filter is used
+ `speed_name` refers to the name of the speed parameters in `IntervalPositionInfo`

+ For the `Kay_ripple_detector` (options are currently Kay and Karlsson, see `ripple_detection` package for specifics) the parameters are:
    + `speed_threshold`: maxmimum speed the animal can move (in cm/s)
    + `minimum_duration`: minimum time above threshold (in seconds)
    + `zscore_threshold`: mimimum value to be considered a ripple (in standard deviations from mean)
    + `smoothing_sigma`: how much to smooth the signal in time (in seconds)
    + `close_ripple_threshold`: exclude ripples closer than this amount (in seconds)



## 3. Make sure the speed for this interval exists

The speed for this interval should exist under the default position parameter set and for a given interval.

In [23]:
(
    IntervalPositionInfo
    & {
        "nwb_file_name": nwb_file_name,
        "position_info_param_name": "default",
        "interval_list_name": "pos 1 valid times",
    }
).fetch1_dataframe()

,head_position_x,head_position_y,head_orientation,head_velocity_x,head_velocity_y,head_speed
time,,,,,,
1.581887e+09,91.051650,211.127050,2.999696,1.387074,2.848838,3.168573
1.581887e+09,90.844337,211.417287,3.078386,3.123201,3.411111,4.624939
1.581887e+09,90.637025,211.707525,-3.114572,5.431643,4.089597,6.799085
1.581887e+09,90.802875,211.596958,-3.033109,8.097753,4.979262,9.506138
1.581887e+09,91.288579,211.482443,-3.062550,10.840482,6.071373,12.424880
...,...,...,...,...,...,...
1.581888e+09,182.158583,201.452467,-0.986926,0.348276,0.218575,0.411182
1.581888e+09,182.158583,201.397183,-0.978610,0.279135,-0.058413,0.285182
1.581888e+09,182.213867,201.341900,-0.957589,0.193798,-0.283200,0.343162


We can see that `head_speed` exists. Remember this must be the same as set in the `RippleParameters`.

## 4. Running Ripple Detection and Fetching Ripple Times

Now we can put everything together.

In [25]:
key = {
    "ripple_param_name": "default",
    "nwb_file_name": nwb_file_name,
    "group_name": "CA1",
    "position_info_param_name": "default",
    "interval_list_name": "pos 1 valid times",
}
RippleTimes().populate(key)

[2022-11-28 14:50:49,354][WARNING]: Skipped checksum for file with hash: 4bfb6228-ab9c-7765-6fec-3a4a84d0747f, and path: /stelmo/nwb/analysis/chimi20200216_new_QZ5RAQKFBU.nwb


Computing ripple times for: {'ripple_param_name': 'default', 'nwb_file_name': 'chimi20200216_new_.nwb', 'group_name': 'CA1', 'position_info_param_name': 'default', 'interval_list_name': 'pos 1 valid times'}
Writing new NWB file chimi20200216_new_52A55DER85.nwb


We can get the ripple times by using the method `fetch1_dataframe`

In [30]:
ripple_times = (RippleTimes() & key).fetch1_dataframe()
ripple_times

,start_time,end_time
id,,
0,1.581887e+09,1.581887e+09
1,1.581887e+09,1.581887e+09
2,1.581887e+09,1.581887e+09
3,1.581887e+09,1.581887e+09
4,1.581887e+09,1.581887e+09
...,...,...
672,1.581888e+09,1.581888e+09
673,1.581888e+09,1.581888e+09
674,1.581888e+09,1.581888e+09
